In [31]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import numpy.linalg as la

fama_french = pd.read_csv('fama_french_5factor.csv',index_col=0)
fama_french.index = pd.date_range('2010-01-01', periods=fama_french.shape[0], freq='M')

R = pd.read_csv('returns_out_of_sample.csv',index_col=0)
R.index = pd.date_range('2010-01-01', periods=R.shape[0], freq='M')

T, N = R.shape

R.head()

Agric  Food   Soda  Beer  Smoke   Toys    Fun  Books  Hshld  \
2010-01-31  -6.64  0.60  -2.42 -3.70  -3.35  -2.32   2.34   3.55   0.85   
2010-02-28  -6.60  3.26  13.70  0.25   4.43  14.52   3.15  -3.01   3.69   
2010-03-31   2.04  4.46   8.20  6.11   6.12   6.57  14.13   5.47   2.13   
2010-04-30 -10.17  0.18  -1.73 -1.76  -2.51  -0.90  15.13   2.31   1.21   
2010-05-31 -18.19 -3.86  -0.22 -3.88  -7.69  -5.69  -7.47 -10.35  -4.35   

            Clths  ...  Boxes  Trans  Whlsl  Rtail  Meals  Banks  Insur  \
2010-01-31  -3.11  ...  -8.64  -4.04  -2.55  -1.92  -0.98   0.44   0.73   
2010-02-28   6.07  ...   7.90   7.25   5.89   4.32   4.50   1.81   3.51   
2010-03-31  11.39  ...   6.06   8.11   5.99   6.14   8.27   8.46   8.10   
2010-04-30   4.23  ...   1.84   3.82   3.20   1.85   7.31   3.05  -1.67   
2010-05-31  -5.67  ...  -8.49  -5.98  -4.01  -5.83  -4.76 -11.00  -5.88   

            RlEst   Fin  Other  
2010-01-31  -5.02 -7.99   3.54  
2010-02-28   7.32  3.91   1.06  
2010-03-31  14.73  6.96   9.82  
2010-04-30   6.05 -1.45   3.81  
2010-05-31 -11.24 -8.15 -10.95  

[5 rows x 49 columns]

In [51]:
R_m = fama_french['Mkt-RF']
StockList = R.columns

# This isn't in the notebook, but the regression equation he gives doesn't make sense to me if I don't do it
for i in StockList:
    R[i] = R[i] - fama_french['RF']

Coefs = pd.DataFrame([la.lstsq(np.vstack((np.ones(T), R_m)).T, R[Stock], rcond=None)[0] for Stock in StockList], 
                     index=StockList, columns=['$\\alpha$','$\\beta$'])

display(Coefs)

$\alpha$   $\beta$
Agric -0.847564  1.111934
Food   0.127198  0.548272
Soda   0.538681  0.461690
Beer   0.396821  0.476605
Smoke  0.270367  0.666330
Toys  -0.202771  1.070275
Fun    0.155283  1.380038
Books -0.724590  1.172820
Hshld  0.093934  0.574096
Clths  0.189957  0.980414
Hlth  -0.291654  1.040385
MedEq  0.159609  0.893905
Drugs  0.105185  0.776615
Chems -0.553125  1.285338
Rubbr -0.113026  1.106171
Txtls -0.498939  1.339005
BldMt -0.470647  1.443510
Cnstr -0.642536  1.325348
Steel -1.643433  1.624452
FabPr -0.710962  1.283432
Mach  -0.524938  1.455155
ElcEq -0.703102  1.351602
Autos -0.754379  1.423486
Aero   0.049033  1.079313
Ships -0.307517  1.707156
Guns   0.662922  0.745700
Gold   0.106564  0.112503
Mines -1.411187  1.554505
Coal  -3.190643  1.418459
Oil   -1.139984  1.204796
Util   0.311897  0.368861
Telcm  0.047342  0.824963
PerSv -0.954003  1.236047
BusSv -0.008891  1.097265
Hardw -0.477853  1.151526
Softw  0.021065  1.036367
Chips  0.072979  1.186808
LabEq  0.074760  1.195867
Paper -0.400891  1.037471
Boxes -0.415784  1.143216
Trans -0.241286  1.104042
Whlsl -0.259362  1.007052
Rtail  0.107338  0.929587
Meals  0.491752  0.666894
Banks -0.344386  1.215703
Insur  0.122625  0.940091
RlEst -0.903825  1.587621
Fin   -0.673496  1.307900
Other -0.230302  0.936283

In [57]:
import statsmodels.formula.api as smf

df = pd.concat([R,fama_french],axis=1)

coefs_3fact = pd.DataFrame(index=StockList,columns=['$\\alpha$','$\\beta_{Mkt-RF}$','$\\beta_{SBM}$','$\\beta_{HML}$'])

for i in StockList:
    res = smf.ols("{} ~ Q('Mkt-RF') + SMB + HML".format(i),df).fit()

    for j in range(len(coefs_3fact.columns)):
        coefs_3fact.loc[i,coefs_3fact.columns[j]] = res.params[j]

coefs_3fact

$\alpha$ $\beta_{Mkt-RF}$ $\beta_{SBM}$ $\beta_{HML}$
Agric   -0.876085          1.13275     -0.080573    -0.0148731
Food    0.0383621         0.620565      -0.29491    -0.0021256
Soda     0.329305         0.612293     -0.578581     -0.122168
Beer     0.217049         0.596961     -0.444056     -0.157919
Smoke  0.00815736          0.87711     -0.855142    -0.0216827
Toys    -0.245601          1.05524      0.151567     -0.296466
Fun      0.131445           1.3083        0.4575     -0.540299
Books   -0.539114            1.055      0.420755      0.200534
Hshld  -0.0102829         0.653689     -0.315254     -0.033389
Clths    0.237911         0.927782      0.239319    -0.0794394
Hlth    -0.166965         0.920909      0.519959     -0.103655
MedEq   0.0981985         0.864057       0.26615     -0.474193
Drugs  -0.0197133         0.805416     0.0142673     -0.434354
Chems   -0.439347          1.22656       0.17864      0.202939
Rubbr   0.0180774         0.997711      0.445651   -0.00734815
Txtls   -0.317478           1.1821       0.65678    -0.0503527
BldMt   -0.216006          1.27328      0.627512      0.225172
Cnstr   -0.395266          1.16421      0.584802      0.243341
Steel    -1.33033          1.46764      0.462438      0.587782
FabPr      -0.247         0.966195         1.185      0.368367
Mach     -0.29708          1.31384      0.496667      0.266703
ElcEq   -0.540296          1.23024      0.474953     0.0697887
Autos    -0.48654          1.26652      0.530002      0.367625
Aero     0.053861          1.07954   -0.00841527     0.0246997
Ships  0.00330877          1.52171      0.634391       0.40719
Guns     0.587342         0.802176      -0.22129    -0.0315932
Gold    0.0958956        0.0963315      0.110926     -0.147441
Mines    -1.22218          1.47631      0.182178      0.452373
Coal     -2.64987          1.10478       1.05096      0.761465
Oil     -0.845562          1.07797       0.31339      0.674874
Util      0.24572         0.418156     -0.192847    -0.0285795
Telcm   0.0143585         0.872537     -0.231573      0.121989
PerSv   -0.748977          1.10832      0.450274      0.236588
BusSv  -0.0117564          1.05664      0.243439     -0.254477
Hardw   -0.558502          1.20611     -0.202701    -0.0673367
Softw   -0.160224           1.1237     -0.247392     -0.360817
Chips  -0.0430233           1.2353     -0.114789      -0.27464
LabEq   0.0839965          1.13246      0.359753     -0.330766
Paper   -0.379386          1.01983     0.0721934  -0.000293323
Boxes    -0.41311           1.1308     0.0691628     -0.060569
Trans   -0.101466           1.0179      0.301421      0.167027
Whlsl   -0.131539         0.916658      0.344105     0.0837515
Rtail    0.019479         0.956733    -0.0305094     -0.264764
Meals     0.45443          0.67848    -0.0132855     -0.112142
Banks   -0.107711          1.13574      0.122453      0.672722
Insur    0.227266         0.892933      0.123647      0.227522
RlEst   -0.747273           1.4854       0.37148      0.152828
Fin     -0.455462          1.22212      0.184173      0.547961
Other    -0.17128         0.967735     -0.272076      0.472119

In [59]:
coefs_5fact = pd.DataFrame(index=StockList,columns=['$\\alpha$','$\\beta_{Mkt-RF}$','$\\beta_{SBM}$','$\\beta_{HML}$','$\\beta_{RMW}$','$\\beta_{CMA}$'])

for i in StockList:
    res = smf.ols("{} ~ Q('Mkt-RF') + SMB + HML + RMW + CMA".format(i),df).fit()

    for j in range(len(coefs_5fact.columns)):
        coefs_5fact.loc[i,coefs_5fact.columns[j]] = res.params[j]

coefs_5fact

$\alpha$ $\beta_{Mkt-RF}$ $\beta_{SBM}$ $\beta_{HML}$ $\beta_{RMW}$  \
Agric     -0.79247           1.1077     -0.181001     -0.020934     -0.440279   
Food    -0.0918091           0.6612     -0.233267     -0.189932      0.280565   
Soda      0.101459         0.683244     -0.460599     -0.429895      0.534198   
Beer      0.041033         0.651397     -0.331224     -0.350472      0.505399   
Smoke    -0.290117         0.969318     -0.661717     -0.343352      0.865942   
Toys     -0.404548          1.10357      0.301584     -0.370109      0.662136   
Fun       0.272149          1.26366      0.432056     -0.251514     -0.127192   
Books    -0.622079           1.0809      0.459564     0.0798359      0.176769   
Hshld    -0.169151         0.703362     -0.244648     -0.272235      0.322634   
Clths    0.0986984         0.969625      0.398583    -0.0858862      0.699084   
Hlth     -0.095311         0.900287      0.385062     -0.210559      -0.58607   
MedEq     0.143051          0.85195      0.135336     -0.637697     -0.565108   
Drugs   -0.0200147          0.80687    -0.0642869     -0.598697     -0.335786   
Chems    -0.444795           1.2276      0.219074      0.273919      0.173571   
Rubbr   -0.0932545          1.03165      0.545707    -0.0693878      0.442317   
Txtls    -0.402916          1.20692      0.804359     0.0494869      0.642093   
BldMt    -0.267516          1.28874      0.687568      0.225132      0.263484   
Cnstr    -0.320256          1.14107      0.533796      0.319311      -0.22787   
Steel     -1.44012          1.50203      0.507518      0.414996      0.207089   
FabPr    -0.187002         0.947207       1.17156      0.486106    -0.0653275   
Mach     -0.334716          1.32497      0.550389      0.287173      0.234592   
ElcEq    -0.597927          1.24823      0.502196      -0.01346      0.124009   
Autos    -0.433608          1.25023       0.49111        0.4152      -0.17319   
Aero    0.00676754          1.09387     0.0349239   0.000571816      0.191441   
Ships   -0.0630104          1.54266        0.6519      0.282564     0.0835319   
Guns      0.585952         0.802116     -0.192088     0.0258537      0.125024   
Gold     -0.160267         0.177973      0.135233     -0.719053      0.137439   
Mines     -1.38094          1.52588      0.257066      0.222698      0.340929   
Coal      -2.67578           1.1154       0.91722      0.419979     -0.568375   
Oil      -0.900321          1.09604      0.283005      0.478574     -0.122735   
Util     0.0907426         0.466739     -0.131301     -0.276845      0.283397   
Telcm    -0.181043         0.934439     -0.191411     -0.269006      0.197269   
PerSv    -0.752541          1.11051      0.389983      0.102358     -0.257283   
BusSv    0.0155479          1.04837      0.215709     -0.245909     -0.122123   
Hardw    -0.518184           1.1919     -0.127624      0.186969      0.315684   
Softw -0.000784825          1.07353     -0.299742    -0.0825604     -0.244668   
Chips   0.00296051          1.21878    -0.0112034     0.0528029      0.436818   
LabEq     0.146627          1.11323      0.311799     -0.278512     -0.213206   
Paper    -0.504796          1.05845       0.16217     -0.117525      0.401068   
Boxes    -0.364077          1.11611     0.0103121    -0.0640349     -0.258008   
Trans    -0.256346          1.06515      0.437974     0.0752193      0.604043   
Whlsl    -0.224151         0.945738       0.37808    -0.0704488      0.157364   
Rtail    -0.126052          1.00137       0.08431     -0.379127      0.509908   
Meals     0.301944         0.724774      0.134507     -0.174724      0.651782   
Banks     0.122834          1.06418    -0.0100275      0.956807     -0.596537   
Insur     0.297498         0.871888     0.0395709       0.22301     -0.368624   
RlEst    -0.687701          1.46657      0.356372      0.266067    -0.0723838   
Fin      -0.274052          1.16576     0.0825027      0.776863     -0.458389   
Other    -0.265681         0.998185     -